Code to make the shapefile for the whole study region and the base topographic layers. Then format the grouped land cover map and convert it into a full length NumPy array for use in model training.

Note that the final grid to iterate over the shapefile, as used in `bucket_to_array` can be found in `Fire_weather_data_comparison/Process_CEMS_and_NASA_fire_weather_data.ipynb`.

## Imports

In [ ]:
import numpy as np
import ee
ee.Authenticate()
ee.Initialize(project='spherical-berm-323321')

In [ ]:
final = ee.FeatureCollection('users/andyc97/model_shapefiles/final_shapefile')
final_grid = ee.FeatureCollection('users/andyc97/model_shapefiles/final_grid')
south_siberia = ee.FeatureCollection('users/andyc97/model_shapefiles/south_siberia_final')
final_north = ee.FeatureCollection('users/andyc97/model_shapefiles/final_north')

## Make final shapefile for whole region

In [ ]:
# Load the ecoregions
ecoRegions = ee.FeatureCollection('RESOLVE/ECOREGIONS/2017')

# Apply filters
biome_filter = ee.Filter.inList('BIOME_NUM', [6, 11])
realm_filter = ee.Filter.inList('REALM', ['Nearctic', 'Palearctic'])
combined_filter = ee.Filter.And(biome_filter, realm_filter)
selected_regions = ecoRegions.filter(combined_filter)

In [ ]:
# Export monthly image to asset - edit as necessary
task = ee.batch.Export.table.toAsset(
    collection=selected_regions,
    description='final_shapefile',
    assetId='users/andyc97/model_preprocessed/final_shapefile',
    maxPixels=1e13)
task.start()

## Base topographic layers

In [ ]:
# TEM Land Cover map - original is EPSG:6931
#tem_map = ee.Image('users/andyc97/model_shapefiles/TEM_LandCover_Map_V3')
tem_map = ee.Image('users/andyc97/model_shapefiles/TEM_LandCover_Map_V3_Grouped_V1')
tem_map = tem_map.updateMask(tem_map.neq(0)).toFloat()
upscaled_tem = tem_map.reproject(crs='EPSG:6931', scale=4000)
final_LandMask = upscaled_tem.multiply(0).add(1).unmask(0).updateMask(upscaled_tem.neq(0))

# Copernicus map for elevation, slope and aspect
# WILL DROP SOME EDGE PIXELS AT 30m WHEN DERIVING SLOPE AND ASPECT
dataset = ee.ImageCollection('COPERNICUS/DEM/GLO30')
elevation = dataset.select('DEM').max().setDefaultProjection(dataset.first().projection())
slope = ee.Terrain.slope(elevation)
aspect = ee.Terrain.aspect(elevation)

upscaled_elevation = elevation.reproject(crs='EPSG:6931', scale=4000)
upscaled_slope = ee.Terrain.slope(upscaled_elevation)
upscaled_aspect = ee.Terrain.aspect(upscaled_elevation)

# Final base landcover image
base_land_final = upscaled_elevation.addBands(upscaled_slope).addBands(upscaled_aspect).addBands(upscaled_tem).updateMask(upscaled_aspect).updateMask(final_LandMask)

In [ ]:
# Export monthly image to asset
task = ee.batch.Export.image.toAsset(
    image=base_land_final,
    description='base_land_final',
    assetId='users/andyc97/model_shapefiles/final_baseland',
    region=final.geometry(),
    scale=4000,
    crs='EPSG:6931',
    maxPixels=1e9)
task.start()

## Refined land cover layer

In [ ]:
loaded_image = ee.Image('users/andyc97/model_shapefiles/TEM_LandCover_Map_V3_Grouped_V1')
upscaled_loaded = loaded_image.updateMask(loaded_image.neq(100)).reproject(crs='EPSG:6931', scale=4000)

tem_map = ee.Image('users/andyc97/model_shapefiles/TEM_LandCover_Map_V3')
tem_map = tem_map.updateMask(tem_map.neq(0)).toFloat()
upscaled_tem = tem_map.reproject(crs='EPSG:6931', scale=4000)

final_LandMask = upscaled_tem.multiply(0).add(1).unmask(0).updateMask(upscaled_tem.neq(0))

base_land = ee.Image('users/andyc97/model_shapefiles/final_baseland')
aspect = base_land.select('aspect').reproject(crs='EPSG:6931', scale=4000)

era5land = ee.ImageCollection("ECMWF/ERA5_LAND/MONTHLY_AGGR").filterDate('2001-01-01', '2001-01-31')
t2m = era5land.select('temperature_2m').mean().reproject(crs='EPSG:6931', scale=4000)

final_image = upscaled_loaded.updateMask(aspect).updateMask(t2m).clip(final).updateMask(final_LandMask).reproject(crs='EPSG:6931', scale=4000)

In [ ]:
# Export monthly image to cloud bucket
task = ee.batch.Export.image.toCloudStorage(
    image=final_image,
    description='tem_g1',
    bucket='clelland_fire_ml',
    fileNamePrefix='upscaled_tem_landcover_map_g1',
    region=final_LandMask.geometry(),
    scale=4000,
    crs='EPSG:6931',
    maxPixels=1e9,
    formatOptions={'cloudOptimized': True}
)
task.start()

## Convert land cover to array

In [38]:
g1_single = np.load('/home/users/clelland/Model/upscaled_tem_g1_single.npy')
len(g1_single)

1471069

In [49]:
X = 204
repeated_g1 = np.tile(g1_single, (X, 1))
len(repeated_g1)

300098076

In [ ]:
np.save('/gws/nopw/j04/bas_climate/users/clelland/model/testing_data/testing_2020_12_land_g1_array.npy', g1_single)